## Create POSCAR

Zincblende ZnO, lattice constant $a=4.579$ Angstrom

In [26]:
from mykit.vasp.poscar import poscar

zZnO = poscar.zincblende("Zn", "O", aLatt=4.579)

In [27]:
# print zZnO to directly see the POSCAR
print(zZnO)

Zincblende ZnO
1.000000
    4.57900000    0.00000000    0.00000000
    0.00000000    4.57900000    0.00000000
    0.00000000    0.00000000    4.57900000
Zn O
4 4
Direct
    0.000000000     0.000000000     0.000000000 #Zn
    0.000000000     0.500000000     0.500000000 #Zn
    0.500000000     0.000000000     0.500000000 #Zn
    0.500000000     0.500000000     0.000000000 #Zn
    0.250000000     0.250000000     0.250000000 #O
    0.250000000     0.750000000     0.750000000 #O
    0.750000000     0.250000000     0.750000000 #O
    0.750000000     0.750000000     0.250000000 #O


In [28]:
# export to POSCAR
zZnO.write()

## Export POTCAR from POSCAR

The atom types in POSCAR can be extracted by `atomTypes` property (in order)

In [29]:
zZnO.atomTypes

['Zn', 'O']

Parse it (expanded by `*`) to `potcar_search` to create an instance to search POTCARs for "Zn" and "O"

In [30]:
from mykit.vasp.potcar import potcar_search

pts = potcar_search(*zZnO.atomTypes, usegw=False) # usegw set True to search PAW with GW suffix only

In [31]:
# print the names of POTCARs to search
pts.names

('Zn', 'O')

*N.B. You need first setup* `VASP_POT_PBE` *environment variable to your local PBE PAW directory to make the following work for you*

Search the PAW PBE direcotry for available "Zn" and "O" POTCARs, together with their `ENMIN`s and `ENMAX`s

In [32]:
pts.search(xc="PBE")

('/Users/stevezhang/software/sci/vasp/vasppot-5.4/potpaw_PBE/',
 {'Zn': {'Zn_GW': (246.143, 328.191),
   'Zn': (207.542, 276.723),
   'Zn_sv_GW': (301.249, 401.665)},
  'O': {'O_GW': (310.976, 414.635),
   'O_s': (212.14, 282.853),
   'O_h': (500.0, 700.0),
   'O_s_GW': (250.998, 334.664),
   'O_GW_new': (325.824, 434.431),
   'O': (300.0, 400.0),
   'O_h_GW': (574.139, 765.519)}})

Export the POTCAR to current directory. 

In [33]:
pts.export(xc="PBE") # return the searching path of PAW PBE

'/Users/stevezhang/software/sci/vasp/vasppot-5.4/potpaw_PBE/'

## Create KPOINTS

In [34]:
from mykit.vasp.kpoints import kpoints

### Gamma-centered kgrids

In [35]:
kpt = kpoints(kgrid=[4,4,4])

In [36]:
# preview the KPOINTS
kpt.print()

KPOINTS generated by mykit
0
Gamma
  4   4   4
  0   0   0


In [37]:
# export to KPOINTS
kpt.write()